In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from abc import ABC, abstractmethod
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder


In [2]:
from barcelona_prediction_lib.barcelona_prediction_lib.data_loader import *
from barcelona_prediction_lib.barcelona_prediction_lib.data_preprocessor import *
from barcelona_prediction_lib.barcelona_prediction_lib.feature_extractor import * 
from barcelona_prediction_lib.barcelona_prediction_lib.model import *

In [69]:
import os
cwd = os.getcwd()
print(cwd)
df = pd.read_csv(cwd + '/train.csv')

c:\Users\Blanca\Dropbox\My PC (Blanca)\Desktop\DSDM\1-MachineLearning\bcn_aparment_price


# SPLIT THE DATASET

In [70]:
loader = DataLoaderSpliter('train.csv')
train_df, test_df = loader.load_and_split()
train_df.head()

,id,num_rooms,num_baths,square_meters,orientation,year_built,door,is_furnished,has_pool,neighborhood,num_crimes,has_ac,accepts_pets,num_supermarkets,price
1467,2580,2.0,1.0,-14.0,north,1968.0,3º - 2a,False,False,Sant Andreu,10.0,True,True,NaN,1074
5768,7129,1.0,1.0,97.0,west,1971.0,2º - 3a,False,False,Sarrià-Sant Gervasi,0.0,False,False,NaN,1079
5714,4083,3.0,1.0,160.0,NaN,1951.0,2º - 2a,True,True,Nou Barris,4.0,True,NaN,NaN,1306
1578,2551,4.0,1.0,116.0,north,2009.0,NaN,True,True,Eixample,0.0,False,True,NaN,1441
6958,4927,1.0,2.0,165.0,west,2002.0,2º - 1a,True,False,Nou Barris,2.0,True,True,NaN,1504


### Number of nulls per column

In [71]:
df.isnull().sum().sort_values(ascending=False)

num_supermarkets    6589
orientation         2341
year_built           170
has_ac               169
is_furnished         165
neighborhood         165
num_baths            160
num_crimes           160
has_pool             156
square_meters        155
accepts_pets         155
door                 149
id                     0
num_rooms              0
price                  0
dtype: int64

### Shape of the dataframes

In [72]:
print('Train shape:', train_df.shape, '\n Test shape:', test_df.shape)

Train shape: (6400, 15) 
 Test shape: (1600, 15)


In [73]:
train_df = train_df.drop(['num_supermarkets','orientation'], axis=1)
test_df = test_df.drop(['num_supermarkets','orientation'], axis=1)

In [74]:
train_df['floor'] = train_df['door'].str.extract(r'(\d{1,2})º')
test_df['floor'] = test_df['door'].str.extract(r'(\d{1,2})º')

# 1. Impute year with neighborhood mode

In [75]:
year_mode_df = train_df.groupby('neighborhood')['year_built'].apply(lambda x: x.mode()).reset_index()
year_mode_df

,neighborhood,level_1,year_built
0,Ciutat Vella,0,1961.0
1,Eixample,0,1979.0
2,Gràcia,0,1950.0
3,Gràcia,1,2001.0
4,Horta,0,1984.0
5,Les Cors,0,1961.0
6,Nou Barris,0,1995.0
7,Sant Andreu,0,1967.0
8,Sant Martí,0,1966.0
9,Sant Martí,1,2018.0


In [76]:
# Agrupar per 'neighborhood' i crear les columnes year1 i year2
year_mode_df['year1'] = year_mode_df.groupby('neighborhood')['year_built'].transform(lambda x: x.sort_values().iloc[0] if len(x) > 0 else None)
year_mode_df['year2'] = year_mode_df.groupby('neighborhood')['year_built'].transform(lambda x: x.sort_values().iloc[1] if len(x) > 1 else None)
year_mode_df['year3'] = year_mode_df.groupby('neighborhood')['year_built'].transform(lambda x: x.sort_values().iloc[2] if len(x) > 2 else None)
print(year_mode_df)


           neighborhood  level_1  year_built   year1   year2   year3
0          Ciutat Vella        0      1961.0  1961.0     NaN     NaN
1              Eixample        0      1979.0  1979.0     NaN     NaN
2                Gràcia        0      1950.0  1950.0  2001.0     NaN
3                Gràcia        1      2001.0  1950.0  2001.0     NaN
4                 Horta        0      1984.0  1984.0     NaN     NaN
5              Les Cors        0      1961.0  1961.0     NaN     NaN
6            Nou Barris        0      1995.0  1995.0     NaN     NaN
7           Sant Andreu        0      1967.0  1967.0     NaN     NaN
8            Sant Martí        0      1966.0  1966.0  2018.0  2020.0
9            Sant Martí        1      2018.0  1966.0  2018.0  2020.0
10           Sant Martí        2      2020.0  1966.0  2018.0  2020.0
11                Sants        0      1968.0  1968.0  2010.0     NaN
12                Sants        1      2010.0  1968.0  2010.0     NaN
13  Sarrià-Sant Gervasi        0  

In [77]:
train_df.groupby('neighborhood')['year_built'].value_counts().reset_index()

,neighborhood,year_built,count
0,Ciutat Vella,1961.0,15
1,Ciutat Vella,1997.0,14
2,Ciutat Vella,2006.0,14
3,Ciutat Vella,1956.0,13
4,Ciutat Vella,1986.0,12
...,...,...,...
715,Sarrià-Sant Gervasi,1998.0,5
716,Sarrià-Sant Gervasi,1967.0,4
717,Sarrià-Sant Gervasi,2013.0,3
718,Sarrià-Sant Gervasi,1984.0,3


In [78]:
year_mode_count = year_mode_df.neighborhood.value_counts().reset_index()
year_mode_count

,neighborhood,count
0,Sant Martí,3
1,Gràcia,2
2,Sants,2
3,Sarrià-Sant Gervasi,2
4,Ciutat Vella,1
5,Eixample,1
6,Horta,1
7,Les Cors,1
8,Nou Barris,1
9,Sant Andreu,1


In [79]:
def fill_year_mode(df):
    for index in range(len(year_mode_count)):
        barri = year_mode_count.loc[index, 'neighborhood']
        if year_mode_count.loc[index, 'count'] == 1:
            row = year_mode_df[year_mode_df['neighborhood'] == barri].iloc[0]
            year = row['year1']
            rows_to_fill = ((df['neighborhood'] == barri) & (df['year_built'].isna()==True))
            df.loc[rows_to_fill, 'year_built'] = year

        elif year_mode_count.loc[index, 'count'] == 2:
            row = year_mode_df[year_mode_df['neighborhood'] == barri].iloc[0]
            year1 = row['year1']
            year2 = row['year2']
            rows_to_fill = ((df['neighborhood'] == barri) & (df['year_built'].isna()==True))
            year_split = np.random.choice([year1, year2], size=sum(rows_to_fill) , p=[0.5,0.5])
            df.loc[rows_to_fill, 'year_built'] = year_split

        elif year_mode_count.loc[index, 'count'] == 3:
            row = year_mode_df[year_mode_df['neighborhood'] == barri].iloc[0]
            year1 = row['year1']
            year2 = row['year2']
            year3 = row['year3']
            rows_to_fill = ((df['neighborhood'] == barri) & (df['year_built'].isna()==True))
            year_split = np.random.choice([year1, year2, year3], size=sum(rows_to_fill) , p=[1/3,1/3,1/3])
            df.loc[rows_to_fill, 'year_built'] = year_split
    return df

In [80]:
train_df = fill_year_mode(train_df)
test_df = fill_year_mode(test_df)

# 2. Fill null Neighborhoods according to Crime

In [81]:
mean_crimes_df = df.groupby('neighborhood')['num_crimes'].agg('mean').round(2).reset_index()
mean_crimes_df = mean_crimes_df.rename(columns={'num_crimes': 'mean_num_crimes'})
mean_crimes_df

,neighborhood,mean_num_crimes
0,Ciutat Vella,3.08
1,Eixample,0.92
2,Gràcia,2.22
3,Horta,2.54
4,Les Cors,1.43
5,Nou Barris,5.12
6,Sant Andreu,4.49
7,Sant Martí,3.50
8,Sants,3.67
9,Sarrià-Sant Gervasi,0.63


In [82]:
mean_crimes_df['diff'] = np.nan

def fill_neighborhood(df):

    df = df.reset_index(drop=True)

    for i in range(len(df)):
        barri = df.iloc[i]['neighborhood']
        unknown_neigh_crime = df.loc[i, 'num_crimes']

        if pd.isna(barri) and pd.notna(unknown_neigh_crime):

            for j in range(len(mean_crimes_df)):
                diff = abs(unknown_neigh_crime - mean_crimes_df.loc[j, 'mean_num_crimes'])
                mean_crimes_df.loc[j, 'diff'] = diff 

            closest = mean_crimes_df.loc[mean_crimes_df['diff'].idxmin(), 'neighborhood']
            df.loc[i, 'neighborhood'] = closest
            
    return df

In [83]:
train_df = fill_neighborhood(train_df)
test_df = fill_neighborhood(test_df)

# Fill Null values in Boolean columns
Fill with random True and False value respecting the proportion for the year and neighbourhood

In [84]:
train_df.columns

Index(['id', 'num_rooms', 'num_baths', 'square_meters', 'year_built', 'door',
       'is_furnished', 'has_pool', 'neighborhood', 'num_crimes', 'has_ac',
       'accepts_pets', 'price', 'floor'],
      dtype='object')

In [85]:
def fill_boolean_nans(df_to_fill,column_to_fill):
    """
    Fills NaN with random 1s and 0s according to proportion for that neightborhood & year.
    df_to_fill = train_df or test_df
    column_to_fill = accepts_pets, has_ac, has_pool, is_furnished
    """
    averages_df = df_to_fill.groupby(['neighborhood','year_built'])[column_to_fill].mean().reset_index()
    averages_df = averages_df.rename(columns={column_to_fill: 'proportion'})
    averages_df['proportion'] = averages_df['proportion'].fillna(0.5)

    for i in range(len(averages_df)):
        barri = averages_df.loc[i, 'neighborhood']
        any = averages_df.loc[i, 'year_built']
        p = averages_df.loc[i, 'proportion']

        rows = ((df_to_fill['neighborhood'] == barri) & 
            (df_to_fill['year_built'] == any) & 
            (df_to_fill[column_to_fill].isna()==True))
    
        df_to_fill.loc[rows,column_to_fill] = np.random.choice([True, False], size=sum(rows), p=[p, 1-p])

    return df_to_fill

In [91]:
def bool_to_int(df):
    """
    Transforms accepts_pets, has_ac, has_pool, and is_furnished into integers.
    Must be used after fill_boolean_nans().
    """
    df.has_pool = df.has_pool.fillna(0)
    df.has_ac = df.has_ac.fillna(0)
    df.is_furnished = df.is_furnished.fillna(0)
    df.accepts_pets = df.accepts_pets.fillna(0)
                                             
    df.has_pool = df.has_pool.astype(int)
    df.has_ac = df.has_ac.astype(int)
    df.is_furnished = df.is_furnished.astype(int)
    df.accepts_pets = df.accepts_pets.astype(int)
    return df

In [87]:
train_df = fill_boolean_nans(train_df, 'has_ac')
train_df = fill_boolean_nans(train_df, 'has_pool')
train_df = fill_boolean_nans(train_df, 'is_furnished')
train_df = fill_boolean_nans(train_df, 'accepts_pets')

In [88]:
test_df = fill_boolean_nans(test_df, 'has_ac')
test_df = fill_boolean_nans(test_df, 'has_pool')
test_df = fill_boolean_nans(test_df, 'is_furnished')
test_df = fill_boolean_nans(test_df, 'accepts_pets')

In [92]:
train_df = bool_to_int(train_df)
test_df = bool_to_int(test_df)

In [93]:
train_df.isnull().sum()

id                 0
num_rooms          0
num_baths        133
square_meters    105
year_built         3
door             115
is_furnished       0
has_pool           0
neighborhood       6
num_crimes       132
has_ac             0
accepts_pets       0
price              0
floor            115
dtype: int64

In [90]:
test_df.isnull().sum()

id                0
num_rooms         0
num_baths        27
square_meters    50
year_built        0
door             34
is_furnished      0
has_pool          0
neighborhood      0
num_crimes       28
has_ac            0
accepts_pets      0
price             0
floor            34
dtype: int64